In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
import collections, re
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
import re
import tensorflow as tf

/Users/mollyperlich/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/mollyperlich/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#read data
data = pd.read_json('News_With_Labels.json')
data.head()

,createdate,articles
0,2019-07-03,"{'publishdate': '1/2/2018', 'content': ' Mor..."
1,2019-07-03,"{'publishdate': '1/2/2018', 'content': 'If Pro..."
2,2019-07-03,"{'publishdate': '1/2/2018', 'content': 'A YouT..."
3,2019-07-03,"{'publishdate': '1/2/2018', 'content': ' Mor..."
4,2019-07-03,"{'publishdate': '1/2/2018', 'content': 'Q. I ..."


In [3]:
#convert to df
df = pd.DataFrame(data)
df.head()

,createdate,articles
0,2019-07-03,"{'publishdate': '1/2/2018', 'content': ' Mor..."
1,2019-07-03,"{'publishdate': '1/2/2018', 'content': 'If Pro..."
2,2019-07-03,"{'publishdate': '1/2/2018', 'content': 'A YouT..."
3,2019-07-03,"{'publishdate': '1/2/2018', 'content': ' Mor..."
4,2019-07-03,"{'publishdate': '1/2/2018', 'content': 'Q. I ..."


In [4]:
stock_df = pd.DataFrame([])

for i in df['articles']:
    stock_df = stock_df.append(pd.DataFrame({'content': i['content'], 'label': i['label']}, index=[0]), ignore_index=True)

print(f"Number of words in raw data: {stock_df['content'].apply(lambda x: len(x.split(' '))).sum()}") 
stock_df.head()

Number of words in raw data: 4827973


,content,label
0,\n\n\nMore on NYTimes.com\n\n\n\n\n,Buy
1,If Procter & Gamble were sponsoring a 2018 adv...,Buy
2,A YouTube star with millions of followers apol...,Buy
3,\n\n\nMore on NYTimes.com\n\n\n\n\n,Buy
4,Q. I see posts in my Twitter feed that have “...,Buy


In [44]:
#keep a count of how many you delete
bad_rows = []
for i in range(0,stock_df.shape[0]):
    #if the number of characters is below 100
    if len(stock_df['content'].iloc[i])<100:
        bad_rows.append(i)

In [45]:
import nltk
nltk.download('stopwords')

#clean up the text
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

#function to clean the text
def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwords from text
    return text
    
#apply function to each row in the dataframe
stock_df['content'] = stock_df['content'].apply(clean_text)
#number of words after clean up
print(f"Number of words AFTER clean up: {stock_df['content'].apply(lambda x: len(x.split(' '))).sum()}") 
stock_df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mollyperlich/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Number of words AFTER clean up: 2906051


,content,label
0,nytimescom,Buy
1,procter gamble sponsoring 2018 advertisement a...,Buy
2,youtube star millions followers apologized mon...,Buy
3,nytimescom,Buy
4,q see posts twitter feed sensitive content lab...,Buy


In [6]:
content_ = []
for item in stock_df["content"]:
    if (item in content_):
        continue
    else:
        content_.append(item)

In [54]:
from sklearn.model_selection import train_test_split

X = stock_df.content
y = stock_df.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [55]:
bagsofwords = [ collections.Counter(re.findall(r'\w+', txt))
            for txt in content_]
bagsofwords[0]

Counter({'nytimescom': 1})

In [56]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(content_) 
print(X.toarray())
print(vectorizer.get_feature_names())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 6 0 ... 0 0 0]]
['00', '000', '0000', '0001', '000154', '00016', '0002', '000200', '00020062008201020122014201620182020office', '000201020152020', '0003', '0004', '000407', '000a', '000acre', '000amonth', '000an', '000and', '000annual', '000any', '000apop', '000apple', '000archived', '000as', '000ayear', '000bad', '000bankers', '000before', '000being', '000berkshires', '000but', '000by', '000changes', '000chinese', '000clever', '000contrary', '000creditmark', '000creditruth', '000creditsam', '000credittom', '000dave', '000delays', '000donor', '000earlier', '000electric', '000elsewhere', '000employee', '000euro', '000fans', '000federal', '000foot', '000for', '000franc', '000given', '000her', '000here', '000his', '000if', '000im', '000imagea', '000imagefully', '000imageinside', '000in', '000investors', '000it', '000its', '000jim', '000job', '000just', '000landscape', '000last', '000lik

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# list of text documents


# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(content_)
# summarize
# print(vectorizer.vocabulary_)
# print(vectorizer.idf_)
# encode document
#vector = vectorizer.transform([text[0]])
# summarize encoded vector
#print(vector.shape)
#print(vector.toarray())

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [29]:
vector = vectorizer.transform([content_[1]])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(1, 159411)
[[0. 0. 0. ... 0. 0. 0.]]


# Bag-of-Words with Keras
The Keras Python library for deep learning also provides tools for encoding text using the bag-of words-model in the Tokenizer class.
As above, the encoder must be trained on source documents and then can be used to encode training data, test data and any other data in the future. The API also has the benefit of performing basic tokenization prior to encoding the words.


In [30]:
from keras.preprocessing.text import Tokenizer
tk = Tokenizer(num_words=2)
texts = content_
tk.fit_on_texts(texts)
# # create the tokenizer
# t = Tokenizer()
# # fit the tokenizer on the documents
# t.fit_on_texts(content_)
# # summarize what was learned
# # print(t.word_counts)
# # print(t.document_count)
# # print(t.word_index)
# # print(t.word_docs)
# # integer encode documents

In [31]:
tk.texts_to_sequences(texts)

[[],
 [],
 [1, 1, 1],
 [],
 [],
 [1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [],
 [1, 1, 1, 1, 1],
 [],
 [],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [],
 [1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1

In [32]:
tk.word_index

{'said': 1,
 'mr': 2,
 'new': 3,
 'would': 4,
 'company': 5,
 'also': 6,
 'like': 7,
 'one': 8,
 'could': 9,
 'percent': 10,
 'trump': 11,
 'year': 12,
 'people': 13,
 'companies': 14,
 'president': 15,
 'last': 16,
 'states': 17,
 'ms': 18,
 'united': 19,
 'years': 20,
 'two': 21,
 'billion': 22,
 'york': 23,
 'time': 24,
 'may': 25,
 'many': 26,
 'business': 27,
 'million': 28,
 'chief': 29,
 'first': 30,
 'american': 31,
 'trade': 32,
 'make': 33,
 'china': 34,
 'executive': 35,
 'even': 36,
 'news': 37,
 'including': 38,
 'according': 39,
 'facebook': 40,
 'much': 41,
 'get': 42,
 'tax': 43,
 'made': 44,
 'deal': 45,
 'government': 46,
 'market': 47,
 '000': 48,
 'former': 49,
 'us': 50,
 'financial': 51,
 'public': 52,
 'data': 53,
 'house': 54,
 'investors': 55,
 'chinese': 56,
 'work': 57,
 'still': 58,
 'state': 59,
 'money': 60,
 'week': 61,
 'since': 62,
 'times': 63,
 'use': 64,
 'take': 65,
 'federal': 66,
 'technology': 67,
 'political': 68,
 'big': 69,
 'way': 70,
 'back'

In [33]:
encoded_docs = tk.texts_to_matrix(texts, mode='count')
print(encoded_docs)

[[ 0.  0.]
 [ 0.  0.]
 [ 0.  3.]
 ...
 [ 0.  0.]
 [ 0.  5.]
 [ 0. 14.]]


In [34]:
print(len(encoded_docs))
# print(len(text[i]))

4864


# Classification Models

In [36]:
train_data_features = vectorizer.fit_transform(content_)

In [58]:
#logisticRegression

from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.3654618473895582
             precision    recall  f1-score   support

        Buy       0.34      0.31      0.32       314
       Hold       0.42      0.47      0.44       361
       Sell       0.32      0.31      0.31       321

avg / total       0.36      0.37      0.36       996



In [49]:
#RandomForest

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

r_forests = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(n_estimators=100)),
               ])
r_forests.fit(X_train, y_train)

y_pred = r_forests.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.36646586345381527
             precision    recall  f1-score   support

        Buy       0.33      0.17      0.23       314
       Hold       0.38      0.73      0.50       361
       Sell       0.33      0.15      0.20       321

avg / total       0.35      0.37      0.32       996



In [43]:
#Keras 

import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

train_size = int(len(stock_df) * .7)
train_posts = stock_df['content'][:train_size]
train_tags = stock_df['label'][:train_size]

test_posts = stock_df['content'][train_size:]
test_tags = stock_df['label'][train_size:]

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

batch_size = 32
epochs = 4

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 3137 samples, validate on 349 samples
Epoch 1/4
3137/3137 [==============================] - 1s 268us/step - loss: 1.1959 - acc: 0.3424 - val_loss: 1.2023 - val_acc: 0.2607
Epoch 2/4
3137/3137 [==============================] - 1s 207us/step - loss: 0.9970 - acc: 0.5142 - val_loss: 1.2070 - val_acc: 0.2865
Epoch 3/4
3137/3137 [==============================] - 1s 199us/step - loss: 0.8594 - acc: 0.6369 - val_loss: 1.1610 - val_acc: 0.3467
Epoch 4/4
3137/3137 [==============================] - 1s 199us/step - loss: 0.7100 - acc: 0.7163 - val_loss: 1.4605 - val_acc: 0.2693


In [59]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

ValueError: Error when checking target: expected activation_2 to have shape (3,) but got array with shape (1,)